In [1]:
import torch
import torch.nn as nn
from torch import tensor
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
from IPython import display
import pylab as pl
from src.data_structures import Instance
from primes import primes
import seaborn as sns
torch.set_default_tensor_type(torch.DoubleTensor)


In [2]:
batch_size = 32
instance = Instance.generate(128,1)


In [3]:
#Primero explorar que tan factible es resolver esto para las polys de 2 dimensiones
polys = torch.zeros((instance.n_items,instance.n_items))

#Hago la matriz simetrica :D
treshold = 0
func = np.random.random
for i in range(len(polys)):
    for j in range(len(polys)):
        if np.random.random() > treshold:
            polys[i,j] = func()
            #polys[j,i] = polys[j,i]
            if i == j:
                polys[i,j] = 0

sns.heatmap(polys)
plt.legend()
plt.savefig("real.png")
plt.clf()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Figure size 640x480 with 0 Axes>

In [4]:
from itertools import combinations
powers_of_two = np.array([ primes[i] for i in range(0,batch_size)])
#Genero los inputs
inputs = list(combinations(range(instance.n_items),2))
def int_to_vec(tupla):
    output = np.zeros(instance.n_items)
    for x in tupla:
        output[x]= 1
    return output

values = []
inputs2 = []
for tupla in inputs:
    if polys[tupla[0],tupla[1]] != 0:
        values.append(polys[tupla[0],tupla[1]])
        inputs2.append(tupla)

tuplas_vects = list(map(int_to_vec,inputs2))


tuplas = inputs2.copy()
inputs = np.array(list(map(int_to_vec,inputs2)))


In [5]:
def vec_to_nums(test_vec):
    test_vec = np.array_split(test_vec,instance.n_items//batch_size)
    test_vec = np.array([np.multiply(chunk,powers_of_two) for chunk in test_vec])
    test_vec = np.array(list(map(lambda x: np.sum(x),test_vec)))
    return test_vec

tuplas_vects = np.array(list(map(vec_to_nums,tuplas_vects)))
inputs = tuplas_vects


In [36]:
entradas = instance.n_items//batch_size
salidas = 1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the layers of your neural network
        hidden = 100
        self.fc1 = nn.Linear(entradas, hidden)
        self.fc3 = nn.Linear(hidden, salidas)
        self.activation = nn.LeakyReLU()
        self.output_activation = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.output_activation(self.fc1(x)) 
        x = self.output_activation(self.fc3(x))
        return x

net = Net()
criterion = nn.L1Loss()
learning_rate = 0.1
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,0.1,5e-3,10000)
train_in = inputs
expected_out = tensor(values)
epoch = 0
train_tensor = torch.tensor(train_in)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [41]:
for epoch in range(1):
    with torch.no_grad():
        salida_predicha = net(train_tensor)

    for i,value in enumerate(salida_predicha):
        a = salida_predicha[i][0]
        b = expected_out[i]
        a.requires_grad = True
        b.requires_grad = True
        loss = criterion(a,b)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #scheduler.step()
        sys.stdout.write(f"\rEntrenando :D {epoch+1} {loss.item()} {optimizer.param_groups[0]['lr']}")
        sys.stdout.flush()

        #a = net(train_tensor).T
        #predicted = np.zeros((instance.n_items,instance.n_items))
        #for i,tupla in enumerate(tuplas):
        #    predicted[tupla[0],tupla[1]] = a[0][i]
        #sns.heatmap(predicted)
        #plt.savefig("predicted.png")
        #plt.clf()
        #sns.heatmap(np.abs(polys-predicted))
        #plt.savefig("difference.png")
        #plt.clf()


Entrenando :D 1 0.11090992006235012 0.1

Entrenando :D 1 0.3361326920492899 0.11111

In [38]:
a = net(tensor(train_in)).T

In [39]:
import seaborn as sns

predicted = np.zeros((instance.n_items,instance.n_items))
for i,tupla in enumerate(tuplas):
    predicted[tupla[0],tupla[1]] = a[0][i]
    #predicted[tupla[1],tupla[0]] = a[0][i]

#plt.plot(test_data, a[0].detach().numpy(), label='Preds')

sns.heatmap(polys)
plt.title(loss.item())
plt.legend()
plt.savefig("real.png")
plt.clf()
plt.title(loss.item())
sns.heatmap(predicted)
plt.savefig("predicted.png")
plt.clf()
plt.title(loss.item())
sns.heatmap(
    np.abs(polys-predicted)
    )
plt.savefig("difference.png")
plt.clf()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Figure size 640x480 with 0 Axes>